# Snorkel

## Introduction

The goal of this lab is to introduce students to the [Snorkel](http://www.snorkel.org) tool and the possibilities of programmatic label generation using the weak-supervised learning paradigm.

In order to use weakly supervised learning to generate labels, it is necessary to create three datasets:

- **train set**: which does not have any labels
- **validation set**: used for hyperparameter optimization, has labels
- **test set**: used only for final model evaluation, has labels

## Labeling functions

The first step will be to load the dataset and split it into a train set and a test set. Since in our set all SMS have a label, we will simulate a weakly supervised learning problem by randomly removing 80% of the labels. Additionally, Snorkel requires numeric labels, so we need to recode the values.

In [1]:
!head data/smsspamcollection.csv

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [2]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('./data/smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])

df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",-1
1,Ok lar... Joking wif u oni...,-1
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",-1


In [3]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

### Simple keyword search

As a first example, we will use a search for the words "check" and "free" in SMS content

In [5]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

The next step is to apply the labeling functions to the train set.

In [9]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 49664.29it/s]


The result of applying the set of labeling functions to the train set is a matrix of size $m \times n$, where $m$ is the number of examples and $n$ is the number of labeling functions. The matrix contains the result of applying each function to each example.

In [10]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       ...,
       [-1, -1],
       [-1,  1],
       [-1, -1]])

In [11]:
df_train.iloc[1,:]

text     Ok lar... Joking wif u oni...
label                               -1
Name: 1, dtype: object

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.

In [12]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")

Coverage for check(): 1.1%
Coverage for free(): 4.6%


Fortunately, Snorkel offers additional tools that allow for deeper analysis of the result of labeling functions.

In [13]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.0,0.0
free,1,[1],0.046209,0.0,0.0


The meaning of each column is as follows:
- `Polarity`: the set of labels returned by the function
- `Coverage`: the percentage of examples for which the function returns a value other than `ABSTAIN`
- Overlaps: the percentage of examples for which at least one other labeling function returned a value
- Conflicts: the percentage of examples for which at least one other labeling function returned a different value

If the train set contained labels, the method would also return:
- `Correct`: the number of correct labels
- `Incorrect`: number of incorrect labels
- `Empirical Accuracy`: the percentage of correct labels

Let's check the examples labeled by the `free()` function as spam

In [14]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

,text,label
1777,Call FREEPHONE 0800 542 0578 now!,-1
2705,FreeMsg: Fancy a flirt? Reply DATE now & join the UKs fastest growing mobile dating service. Msgs rcvd just 25p to optout txt stop to 83021. Reply DATE now!,-1
3727,No chikku nt yet.. Ya i'm free,-1
1423,Congratulations ur awarded either £500 of CD gift vouchers & Free entry 2 our £100 weekly draw txt MUSIC to 87066 TnCs www.Ldew.com 1 win150ppmx3age16,-1
9,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,-1
5492,Marvel Mobile Play the official Ultimate Spider-man game (£4.50) on ur mobile right now. Text SPIDER to 83338 for the game & we ll send u a FREE 8Ball wallpaper,-1
1640,FreeMsg:Feelin kinda lnly hope u like 2 keep me company! Jst got a cam moby wanna c my pic?Txt or reply DATE to 82242 Msg150p 2rcv Hlp 08712317606 stop to 82242,-1
870,What do U want for Xmas? How about 100 free text messages & a new video phone with half price line rental? Call free now on 0800 0721072 to find out more!,-1
2145,FreeMsg: Hey - I'm Buffy. 25 and love to satisfy men. Home alone feeling randy. Reply 2 C my PIX! QlynnBV Help08700621170150p a msg Send stop to stop txts,-1
385,"Double mins and txts 4 6months FREE Bluetooth on Orange. Available on Sony, Nokia Motorola phones. Call MobileUpd8 on 08000839402 or call2optout/N9DX",-1


It seems that the phrase "call now" is also a good indicator for spam. So let's add one more labeling function.

In [15]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 54334.72it/s]


In [16]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.000000,0.0
free,1,[1],0.046209,0.000673,0.0
call_now,2,[1],0.003589,0.000673,0.0


Let's see which examples were labeled as spam by the `call_now()` function but omitted by `free()`.

In [17]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
buckets

{(-1, -1): array([   0,    1,    2, ..., 4454, 4455, 4457]),
 (1,
  -1): array([   6,    8,   42,   58,   68,   74,  111,  138,  142,  143,  148,
         173,  192,  205,  206,  226,  276,  297,  308,  323,  354,  384,
         406,  443,  456,  466,  491,  519,  531,  592,  623,  637,  689,
         711,  750,  789,  794,  843,  865,  901,  924,  947, 1005, 1071,
        1126, 1143, 1152, 1193, 1204, 1217, 1223, 1248, 1273, 1287, 1300,
        1338, 1347, 1393, 1399, 1405, 1460, 1495, 1503, 1512, 1548, 1554,
        1566, 1575, 1583, 1609, 1633, 1637, 1663, 1674, 1694, 1709, 1737,
        1824, 1832, 1840, 1841, 1874, 1880, 1889, 1933, 1971, 2035, 2049,
        2056, 2093, 2109, 2127, 2147, 2159, 2161, 2241, 2277, 2285, 2301,
        2304, 2318, 2361, 2379, 2386, 2397, 2494, 2498, 2500, 2511, 2539,
        2565, 2605, 2646, 2649, 2658, 2678, 2696, 2718, 2727, 2730, 2731,
        2741, 2788, 2856, 2910, 2924, 2941, 2952, 2968, 2998, 3008, 3010,
        3043, 3073, 3077, 3098, 3116, 31

In [21]:
df_train.iloc[buckets[(ABSTAIN, SPAM)]]

,text,label
463,"UpgrdCentre Orange customer, you may now claim your FREE CAMERA PHONE upgrade for your loyalty. Call now on 0207 153 9153. Offer ends 26th July. T&C's apply. Opt-out available",-1
495,Are you free now?can i call now?,-1
797,"Orange customer, you may now claim your FREE CAMERA PHONE upgrade for your loyalty. Call now on 0207 153 9996. Offer ends 14thMarch. T&C's apply. Opt-out availa",-1


In [22]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.000000,0.0
free,1,[1],0.046209,0.000673,0.0
call_now,2,[1],0.003589,0.000673,0.0


#### assignment

Write a labeling function that marks as spam all messages containing the word "HOT" written in capitals.

In [26]:
@labeling_function()
def hot(sms):
    return SPAM if "HOT" in sms.text else ABSTAIN

### Searching based on a regular expression

Another type of labeling function is one that uses regexp to find specific expressions.

In [33]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"((I)|(she)|(he))\s.*\sfree", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, hot]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 19399.64it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.000000,0.000000
free,1,[1],0.046209,0.046209,0.008748
call_now,2,[1],0.003589,0.001346,0.000000
regex_I_am_free,3,"[0, 1]",0.046209,0.046209,0.008748
hot,4,[1],0.001122,0.000673,0.000000


Let's compare examples that the `free()` function labels as spam and the `regex_I_am_free()` function considers valid.

In [34]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)

,text,label
139,You'll not rcv any more msgs from the chat svc. For FREE Hardcore services text GO to: 69988 If u get nothing u must Age Verify with yr network & try again,-1
4728,"Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call",-1
178,"Turns out my friends are staying for the whole show and won't be back til ~ &lt;#&gt; , so feel free to go ahead and smoke that $ &lt;#&gt; worth",-1
3493,I will reach office around &lt;DECIMAL&gt; . &amp; my mobile have problem. You cann't get my voice. So call you asa i'll free,-1
4151,I only work from mon to thurs but Sat i cant leh... Booked liao... Which other day u free?,-1
4386,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 8000930705,-1
3692,"Sir, i am waiting for your call, once free please call me.",-1
5460,December only! Had your mobile 11mths+? You are entitled to update to the latest colour camera mobile for Free! Call The Mobile Update Co FREE on 08002986906,-1
3409,Do you want a New Nokia 3510i colour phone DeliveredTomorrow? With 300 free minutes to any mobile + 100 free texts + Free Camcorder reply or call 08000930705,-1
3951,I got to video tape pple type in message lor. U so free wan 2 help me? Hee... Cos i noe u wan 2 watch infernal affairs so ask u along. Asking shuhui oso.,-1


#### assignment

Write a labeling function that will mark as spam all messages containing any amounts specified with a currency symbol ($99, £1.50)

In [35]:
@labeling_function()
def contains_money(sms):
    return SPAM if re.search(r"\$\d+(\.\d{2})?", sms.text) else ABSTAIN


### Searching based on heuristics

A simple heuristic to find spam is to assume that if more than 10% of the message text is written in capitals, there is a good chance it is spam.

In [37]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, hot, contains_money]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 9484.64it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.001795,0.000000
free,1,[1],0.046209,0.046209,0.008748
call_now,2,[1],0.003589,0.003140,0.000000
regex_I_am_free,3,"[0, 1]",0.046209,0.046209,0.008748
has_many_uppercase_words,4,[1],0.179004,0.019964,0.002019
hot,5,[1],0.001122,0.000897,0.000000
contains_money,6,[1],0.001346,0.000449,0.000000


#### assignment

Write a labeling function that marks as valid those messages that are shorter than 10 words and do not contain any word written in capitals.

In [38]:
@labeling_function()
def short_and_no_uppercase(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return HAM if len(sms.text.split()) < 10 and percentage_uppercase == 0 else ABSTAIN

### Using an external statistical model

When labeling data, you can use external models whose response can be important information for deciding how to label an example. Snorkel has several built-in integrations in the form of the `Preprocessor` interface, in the example below we will use the `SpaCy` library to perform additional grammatical analysis of the text. However, you will need to download the English language model.

In [60]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 4.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [61]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.4) =================
ℹ spaCy installation:
/Users/luncenok/Studia/sem4/DM/.venv/lib/python3.11/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.7.2,<3.8.0   3.7.1   ✔
en_core_web_lg   >=3.7.2,<3.8.0   3.7.1   ✔



In [62]:
import spacyP

nlp = spacyP.load('en_core_web_lg')

In [70]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

# for e in doc.ents:
#     print(e.text, e.label_)
for token in doc:
    print(token.text, token.pos_, token.tag_, token.lemma_)

I PRON PRP I
do AUX VBP do
n't PART RB not
England PROPN NNP England
is AUX VBZ be
a DET DT a
country NOUN NN country
that PRON WDT that
is AUX VBZ be
part NOUN NN part
of ADP IN of
the DET DT the
United PROPN NNP United
Kingdom PROPN NNP Kingdom
. PUNCT . .

 SPACE _SP 

It PRON PRP it
shares VERB VBZ share
land NOUN NN land
borders NOUN NNS border
with ADP IN with
Wales PROPN NNP Wales
to ADP IN to
its PRON PRP$ its
west NOUN NN west
and CCONJ CC and
Scotland PROPN NNP Scotland
to ADP IN to
its PRON PRP$ its
north NOUN NN north
. PUNCT . .

 SPACE _SP 

The DET DT the
Irish PROPN NNP Irish
Sea PROPN NNP Sea
lies VERB VBZ lie
northwest ADV RB northwest
of ADP IN of
England PROPN NNP England
and CCONJ CC and
the DET DT the
Celtic PROPN NNP Celtic
Sea PROPN NNP Sea
to ADP IN to
the DET DT the
southwest NOUN NN southwest
. PUNCT . .

 SPACE _SP 

England PROPN NNP England
is AUX VBZ be
separated VERB VBN separate
from ADP IN from
continental ADJ JJ continental
Europe PROPN NNP Europe
by 

In [71]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacyP = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Assume that short text messages in which a reference to a specific person appears are not spam.

In [72]:
df_train.columns

Index(['text', 'label'], dtype='object')

In [73]:
@labeling_function(pre=[spacyP])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [74]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, hot, contains_money, short_and_no_uppercase]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:47<00:00, 94.04it/s] 


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.003365,0.001570
free,1,[1],0.046209,0.046209,0.012113
call_now,2,[1],0.003589,0.003365,0.000449
regex_I_am_free,3,"[0, 1]",0.046209,0.046209,0.012113
has_many_uppercase_words,4,[1],0.179004,0.026469,0.008524
has_person,5,[0],0.050920,0.030956,0.008748
hot,6,[1],0.001122,0.000897,0.000000
contains_money,7,[1],0.001346,0.000449,0.000000
short_and_no_uppercase,8,[0],0.310229,0.026245,0.004038


Another example of pre-processing data for labeling would be determining the average word frequency of a document. Below we define a function that determines the average word frequency and we decorate it as an example of a pre-processor. When a text message is sent to the next labeling function, the pre-processor will populate the text message with the average word frequency and, based on that, the labeling function will make a decision (we assume that if the text message contains many rare words then it is spam).

In [75]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [76]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [77]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words, hot, contains_money, short_and_no_uppercase]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:01<00:00, 4141.06it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.003589,0.001570
free,1,[1],0.046209,0.046209,0.012113
call_now,2,[1],0.003589,0.003365,0.000449
regex_I_am_free,3,"[0, 1]",0.046209,0.046209,0.012113
has_many_uppercase_words,4,[1],0.179004,0.045312,0.008524
has_person,5,[0],0.050920,0.032974,0.017048
many_rare_words,6,[1],0.063257,0.053163,0.028712
hot,7,[1],0.001122,0.000897,0.000000
contains_money,8,[1],0.001346,0.000449,0.000000
short_and_no_uppercase,9,[0],0.310229,0.045087,0.029161


In [78]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

,text,label
3763,FREE for 1st week! No1 Nokia tone 4 ur mob every week just txt NOKIA to 8007 Get txting and tell ur mates www.getzed.co.uk POBox 36504 W45WQ norm150p/tone 16+,-1
3450,Sorry. || mail? ||,-1
2966,Not yet had..ya sapna aunty manege y'day hogidhe..chinnu full weak and swalpa black agidhane..,-1
2897,"pdate_Now - Double mins and 1000 txts on Orange tariffs. Latest Motorola, SonyEricsson & Nokia & Bluetooth FREE! Call MobileUpd8 on 08000839402 or call2optout/!YHL",-1
2834,"You've won tkts to the EURO2004 CUP FINAL or £800 CASH, to collect CALL 09058099801 b4190604, POBOX 7876150ppm",-1
2216,Prabha..i'm soryda..realy..frm heart i'm sory,-1
4299,"WELL DONE! Your 4* Costa Del Sol Holiday or £5000 await collection. Call 09050090044 Now toClaim. SAE, TCs, POBox334, Stockport, SK38xh, Cost£1.50/pm, Max10mins",-1
2290,Had your mobile 11mths ? Update for FREE to Oranges latest colour camera mobiles & unlimited weekend calls. Call Mobile Upd8 on freefone 08000839402 or 2StopTx,-1
5095,K.k.this month kotees birthday know?,-1
3394,Ok thanx...,-1


#### assignment

Write a labeling function that marks messages containing more than 3 adjectives as spam. Use the SpaCy library for pre-processing. 

__Hint__: the following example shows how to read the part-of-speech label for each token from the message being analyzed. For information on all token properties recognized by SpaCy, see [API documentation](https://spacy.io/api/token)

In [79]:
import spacyP

nlp = spacyP.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

Yetunde    PROPN      NNP        Yetunde   
,          PUNCT      ,          ,         
i          PRON       PRP        I         
'm         AUX        VBP        be        
sorry      ADJ        JJ         sorry     
but        CCONJ      CC         but       
moji       ADJ        JJ         moji      
and        CCONJ      CC         and       
i          PRON       PRP        I         
seem       VERB       VBP        seem      
too        ADV        RB         too       
busy       ADJ        JJ         busy      
to         PART       TO         to        
be         AUX        VB         be        
able       ADJ        JJ         able      
to         PART       TO         to        
go         VERB       VB         go        
shopping   NOUN       NN         shopping  
.          PUNCT      .          .         


## Combining labeling functions into a single model

The goal of labeling functions is not to achieve individually large coverage. Labeling functions are inherently noisy and can make many individual errors. The true utility of labeling functions becomes apparent when multiple functions are combined to form a single model.

We will first build a simple model based on majority voting, and then build a more complex model. 

In [82]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words, hot, contains_money, short_and_no_uppercase, has_many_uppercase_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 1114/1114 [00:00<00:00, 13476.28it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010767,0.003589,0.001570
free,1,[1],0.046209,0.046209,0.012113
call_now,2,[1],0.003589,0.003365,0.000449
regex_I_am_free,3,"[0, 1]",0.046209,0.046209,0.012113
has_person,4,[0],0.050920,0.032974,0.017048
many_rare_words,5,[1],0.063257,0.053163,0.028712
hot,6,[1],0.001122,0.000897,0.000000
contains_money,7,[1],0.001346,0.000449,0.000000
short_and_no_uppercase,8,[0],0.310229,0.045087,0.029161
has_many_uppercase_words,9,[1],0.179004,0.045312,0.008524


In [83]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010772,0.006284,0.003591
free,1,[1],0.052962,0.052962,0.014363
call_now,2,[1],0.005386,0.005386,0.000000
regex_I_am_free,3,"[0, 1]",0.052962,0.052962,0.014363
has_person,4,[0],0.043986,0.026930,0.010772
many_rare_words,5,[1],0.063734,0.053860,0.026032
hot,6,[1],0.001795,0.001795,0.000000
contains_money,7,[1],0.003591,0.000000,0.000000
short_and_no_uppercase,8,[0],0.304309,0.045781,0.029623
has_many_uppercase_words,9,[1],0.202873,0.049372,0.008079


In [103]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [104]:
preds_train

array([-1, -1,  1, ..., -1, -1,  0])

In [105]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

LABEL: -1, count: 2129
LABEL: 0, count: 1363
LABEL: 1, count: 966


In [106]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.083]
INFO:root:[100 epochs]: TRAIN:[loss=0.005]
 36%|███▋      | 182/500 [00:00<00:00, 1816.80epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.003]
INFO:root:[300 epochs]: TRAIN:[loss=0.003]
100%|██████████| 500/500 [00:00<00:00, 1917.81epoch/s]
INFO:root:Finished Training


In [108]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority voting accuracy: 61.4%
Probabilistic model accuracy: 62.2%


Unfortunately, some data points will not receive any label. It is necessary to filter out these points before sending the labeling result for further processing.

In [109]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

((4458, 2), (2483, 2))

As you can see, we were able to quickly prepare labels for about 650 examples (recall that initially no example in the `df_train` set had labels).

The next step will use prepared labels as training data for the actual classifier. We will use simple [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), first pre-processing the input data. Since we are working with text, we will use the [word vector representation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) created based on 5-grams by `CountVectorizer`.

In [110]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import CountVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [111]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0)

In [112]:
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

Logistic regression accuracy: 68.8%


As can be seen, the final model improved the score over the majority vote and the `LabelModel` model.

#### assignment

Complete the above calls with functions that you wrote yourself and check whether your functions improve the quality of the model.

## Transforming functions

The idea of a transforming function is to perform an atomic transformation of an instance. For data that is an image, typical transformations include cropping, rotating, and changing the color palette. For text data, you can replace words with synonyms, substitute named entities, cut random pieces of text, etc. In the following example we will find types of named entities occurring in the text, and then prepare a simple transformer that will randomly replace occurrences of the `PERSON` entity

In [113]:
import spacy

nlp = spacy.load('en_core_web_lg')

for doc in nlp.pipe(df_train.text.sample(frac=0.05)):
    print(f"Entities: {[(e.text, e.label_) for e in doc.ents]}")

Entities: []
Entities: [('2', 'CARDINAL'), ('09058091870', 'CARDINAL'), ('3UZ', 'CARDINAL')]
Entities: [('first', 'ORDINAL')]
Entities: [('one', 'CARDINAL')]
Entities: []
Entities: []
Entities: []
Entities: []
Entities: [('2', 'CARDINAL'), ('09058094594', 'DATE')]
Entities: []
Entities: [('Cos da ge', 'PERSON'), ('wat', 'PERSON')]
Entities: []
Entities: []
Entities: [('U. Todays', 'ORG'), ('2000', 'MONEY'), ('09066358361', 'CARDINAL'), ('12hrs', 'CARDINAL')]
Entities: [('neva', 'GPE'), ('wat', 'PERSON')]
Entities: []
Entities: [('ya babe x', 'PERSON')]
Entities: [('07090298926', 'CARDINAL'), ('Ref:9307622', 'PRODUCT')]
Entities: []
Entities: []
Entities: [('4', 'CARDINAL')]
Entities: [('one', 'CARDINAL'), ('Monday', 'DATE')]
Entities: []
Entities: []
Entities: [('STIL', 'PERSON'), ('4', 'CARDINAL'), ('630', 'CARDINAL')]
Entities: []
Entities: [('2', 'CARDINAL'), ('09058095107', 'PERSON'), ('POBox', 'PERSON'), ('7', 'CARDINAL'), ('S3XY', 'ORG')]
Entities: []
Entities: []
Entities: []
En

In [114]:
person_entities = []

for doc in nlp.pipe(df_train.text):
    for e in doc.ents:
        if e.label_ == 'PERSON':
            person_entities.append(e.text)
        
person_entities[:10]

['Melle Melle',
 'Oru Minnaminunginte Nurungu Vettam',
 'Callertune',
 'KL341',
 '81010 T&C',
 'Ffffffffff',
 'Mark',
 'Abiola',
 'tyler',
 'Mark']

In [115]:
from snorkel.augmentation import transformation_function
from snorkel.preprocess.nlp import SpacyPreprocessor

spacyP = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@transformation_function(pre=[spacyP])
def random_person_ner(sms):
    person_ners = [e.text for e in sms.doc.ents]
    
    if person_ners:
        person_to_replace = np.random.choice(person_ners)
        person_to_add = np.random.choice(person_entities)
        sms.text = sms.text.replace(person_to_replace, person_to_add)
    return sms

Another example of transformation could be using WordNet to find synonyms for words. However, this requires downloading a corpus of data

In [116]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/luncenok/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [117]:
def get_synonym(word):
    
    synsets = wordnet.synsets(word)
    
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        
        return np.random.choice([w.replace("_", " ") for w in words])


In [118]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        synonym = get_synonym(words[word_idx])
        if synonym:
            words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms

Let us now compare the original text message content with the transformed versions.

In [119]:
# source: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/utils.py

from collections import OrderedDict

def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                
    return pd.DataFrame(transformed_examples)


In [120]:
tfs = [random_person_ner, replace_words_with_synonym]

preview_tfs(df_train.sample(frac=0.1), tfs)

,TF Name,Original Text,Transformed Text
0,random_person_ner,You know there is. I shall speak to you in &lt;#&gt; minutes then,You know there is. I shall speak to you in &lt;#&gt; mayb then
1,random_person_ner,yay! finally lol. i missed our cinema trip last week :-(,yay! finally lol. i missed our cinema trip Rakhesh :-(
2,random_person_ner,Love has one law; Make happy the person you love. In the same way friendship has one law; Never make ur friend feel alone until you are alive.... Gud night,Love has Jerry law; Make happy the person you love. In the same way friendship has Jerry law; Never make ur friend feel alJerry until you are alive.... Gud night
3,random_person_ner,What do u want when i come back?.a beautiful necklace as a token of my heart for you.thats what i will give but ONLY to MY WIFE OF MY LIKING.BE THAT AND SEE..NO ONE can give you that.dont call me.i will wait till i come.,What do u want when i come back?.a beautiful necklace as a token of my heart for you.thats what i will give but ONLY to MY WIFE OF MY LIKING.BE THAT AND SEE..NO ONE can give you that.dont call me.i will wait till i come.
4,random_person_ner,I wont. So wat's wit the guys,I wont. So wat's wit the guys
...,...,...,...
887,replace_words_with_synonym,Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..,Your persuasion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. non matured 8. Stylish 9. Simple Pls reply..
888,replace_words_with_synonym,I have to take exam with in march 3,I have to take test with in Mar 3
889,replace_words_with_synonym,"Open rebtel with firefox. When it loads just put plus sign in the user name place, and it will show you two numbers. The lower number is my number. Once you pick that number the pin will display okay!","Open rebtel with firefox. When it loads just put plus sign in the user name place, and IT will show you two numbers. The lower number is my number. Once you pick that number the pin will display okay!"
890,replace_words_with_synonym,"Dear, will call Tmorrow.pls accomodate.","Dear, volition phone call Tmorrow.pls accomodate."


Applying transforming functions requires some policy defining the order and number of transformations. In the example below, two transformation functions are drawn at random and this sequence of two functions is applied twice to each data point. As a result, we triple the size of the train set.

In [121]:
from snorkel.augmentation import RandomPolicy, PandasTFApplier

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

100%|██████████| 446/446 [00:07<00:00, 58.44it/s]


In [122]:
df_train_sample.shape, df_train_augmented.shape

((446, 2), (1338, 2))

#### assignment

Modify the transforming function ``replace_words_with_synonym()`` so that you can restrict the replacement of words with synonyms only for specific parts of speech (e.g., replace only nouns or verbs).